<a href="https://colab.research.google.com/github/Byeongjuuuuu/nlp/blob/main/Ko_MRC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/[GOORM]NLP/Project/Machine Reading Comprehension/7th-goorm-project-2-korean-mrc.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/[GOORM]NLP/Project/Machine Reading Comprehension/7th-goorm-project-2-korean-mrc.zip
  inflating: baseline.csv            
  inflating: blank.csv               
  inflating: test.json               
  inflating: train.json              


In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 929.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_dataset, load_metric#, list_metrics

from transformers import AutoTokenizer, BertConfig, BertPreTrainedModel, AutoModelForQuestionAnswering
from statistics import mean

from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup
from transformers.modeling_outputs import QuestionAnsweringModelOutput
import os
import torch
from torch import nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
import torch.nn.functional as F
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import numpy as np
from tqdm import tqdm, tqdm_notebook
import warnings
import time
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
warnings.filterwarnings(action='ignore')

In [ ]:
from typing import List, Tuple, Dict, Any
import json
import random

class KoMRC:
    def __init__(self, data, indices: List[Tuple[int, int, int]]):
        self._data = data
        self._indices = indices

    # Json을 불러오는 메소드
    @classmethod
    def load(cls, file_path: str):
        with open(file_path, 'r', encoding='utf-8') as fd:
            data = json.load(fd)

        indices = []
        for d_id, document in enumerate(data['data']):
            for p_id, paragraph in enumerate(document['paragraphs']):
                for q_id, _ in enumerate(paragraph['qas']):
                    indices.append((d_id, p_id, q_id))
        
        return cls(data, indices)

    # 데이터 셋을 잘라내는 메소드
    @classmethod
    def split(cls, dataset, eval_ratio: float=.1, seed=42):
        indices = list(dataset._indices)
        random.seed(seed)
        random.shuffle(indices)
        train_indices = indices[int(len(indices) * eval_ratio):]
        eval_indices = indices[:int(len(indices) * eval_ratio)]

        return cls(dataset._data, train_indices), cls(dataset._data, eval_indices)

    def __getitem__(self, index: int) -> Dict[str, Any]:
        d_id, p_id, q_id = self._indices[index]
        paragraph = self._data['data'][d_id]['paragraphs'][p_id]

        
        qa = paragraph['qas'][q_id]

        guid = qa['guid']
        question = qa['question']
        answers = qa['answers'][0]
        context = paragraph['context']
        return {
            'guid': guid,
            'context': context,
            'question': question,
            'answers': answers
        }

    def __len__(self) -> int:
        return len(self._indices)


In [ ]:
dataset = KoMRC.load('./train.json')
print("Number of Samples:", len(dataset))
print(dataset[0])

Number of Samples: 12037
{'guid': '798db07f0b9046759deed9d4a35ce31e', 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.', 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?', 'answers': {'text': '한 달가량', 'answer_start': 478}}


In [ ]:
train_dataset, dev_dataset = KoMRC.split(dataset)
print("Number of Train Samples:", len(train_dataset))
print("Number of Dev Samples:", len(dev_dataset))
print(dev_dataset[0])

Number of Train Samples: 10834
Number of Dev Samples: 1203
{'guid': '844e22ab28924c1697d5ac28801b34c1', 'context': '지난해 주요 연극상을 나눠 가졌던 세 편의 작품이 올봄에 나란히 앙코르 무대를 갖는다. 대한민국연극대상 연기·무대예술상, 동아연극상 작품·희곡·연기상 등을 수상한 ‘알리바이 연대기’(17~20일 대학로 아르코예술극장 대극장, 25일~5월11일 서계동 국립극단 백성희장민호극장), 연극대상에서 대상과 희곡상을 받은 ‘여기가 집이다’(18일~5월22일 대학로 연우소극장), 연극대상 작품·연출상과 김상열연극상 수상작인 ‘황금용’(5월9~18일 서강대 메리홀 대극장)이다. 초연 당시 짧은 상연 기간과 낮은 인지도 등으로 공연을 놓친 연극팬에겐 평단으로부터 작품성을 인정받은 수작을 관람할 수 있는 기회다. ‘알리바이 연대기’는 희곡을 쓰고 연출한 김재엽의 가족사에 근거한 다큐멘터리 드라마다. 1930년에 태어난 한 개인의 사적인 연대기를 바탕으로 그 사이를 파고드는 역사적 순간들을 정밀하게 조명한다. 연출가는 “공적인 권력이 사적인 권리를 지켜주기보다 억압하기 일쑤였던 한국 현대사 속에서 개인은 언제나 무죄를 입증하며 하루하루 자신을 지켜내야 하는 ‘알리바이의 연대기’ 속에서 살아왔다”고 말한다.한국연극평론가협회는 이 작품을 ‘2013년 올해의 연극 베스트3’로 선정하며 “촘촘하고 세세하게 삶에 천착해 개인과 역사에 대한 이분법적 관점을 극복한다. 정치극에 대한 새로운 가능성을 보여줬다”고 평했다. 이 작품으로 연기상을 휩쓴 남명렬을 비롯해 지춘성 정원조 등 초연 배우들이 그대로 출연한다.‘여기가 집이다’는 허름하고 볼품 없는 ‘20년 전통’의 고시원에 모여 사는 사람들의 절망과 희망을 그린 작품. ‘차력사와 아코디언’ ‘택배 왔어요’를 만든 극단 이와삼의 장우재 대표가 직접 대본을 쓰고 연출했다. 나름의 규칙을 가지고 평화로웠던 고시원에 새로운 주인으로 등장한 ‘2

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataset):
        self.question, self.context, self.answer_start, self.answer_text = self.make_dataset(dataset)

    def make_dataset(self, dataset):
        context, question, answer_start, answer_text = [], [], [], []
        for i, data in enumerate(dataset) :
          answer_start.append(data['answers']['answer_start'])
          answer_text.append(data['answers']['text'])
          context.append(data['context'])
          question.append(data['question'])
        return question, context, answer_start, answer_text
        
    def __len__(self):
        return len(self.question)

    def __getitem__(self, idx):
        return self.question[idx], self.context[idx], self.answer_start[idx], self.answer_text[idx]


In [ ]:
train_dataset_custum = CustomDataset(train_dataset)
dev_dataset_custum = CustomDataset(dev_dataset)

In [ ]:
def custom_collate_fn(batch):
    global tokenizer
    question_list, context_list, answer_start, answer_text, after_list = [], [], [], [], []

    for _question, _context, _start, _text in batch:
        question_list.append(_question)
        context_list.append(_context)
        after_list.append(_context[:_start])
        answer_start.append(_start)
        answer_text.append(_text)

    tensorized_input = tokenizer(    
        question_list, context_list,
        add_special_tokens=True,
        padding="longest",  
        max_length=512,
        truncation="only_second",
        return_tensors='pt'
    )

    # answer_start token의 위치를 찾기 위해 list로 반환
    label_tokens = tokenizer(
        question_list, after_list,
        add_special_tokens=False,
        return_tensors=None
    ).input_ids

    # answer text token만 변환
    answer_tokens = tokenizer(
        answer_text,
        add_special_tokens=False,
        return_tensors=None
    ).input_ids

    start = []
    end = []
    for label, answer in zip(label_tokens,answer_tokens):
        if len(label) + len(answer) + 1 <= 512:
            start.append(len(label) + 2)
            end.append(len(label) + len(answer) + 1)
        else :
            start.append(0)
            end.append(0)
    start = torch.LongTensor(start)
    end = torch.LongTensor(end)
    return tensorized_input, start, end, answer_text

In [ ]:
def make_dataloader(dataset, batch_size, s='train') :
  dataloader = DataLoader(
      dataset,
      batch_size =batch_size,
      shuffle = True if s == 'train' else False,
      collate_fn = custom_collate_fn
  )
  print(f'batch_size : {batch_size}')
  return dataloader

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "monologg/kobigbird-bert-base",
    use_fast=True # Whether or not to try to load the fast version of the tokenizer.
    )

In [ ]:
batch_size = 64
accumulation = 32
train_loader = make_dataloader(train_dataset_custum, batch_size//accumulation , 'train')
dev_loader = make_dataloader(dev_dataset_custum, batch_size//accumulation,False)

batch_size : 2
batch_size : 2


In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("monologg/kobigbird-bert-base")

Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'bert.pooler.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.pooler.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BigBirdForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of 

In [ ]:
batch, start, end, answer_text = next(iter(train_loader))
input_ids = batch.input_ids.to(device)
attention_mask = batch.attention_mask.to(device)
token_type_ids = batch.token_type_ids.to(device)        
start = start.to(device)
end = end.to(device)
# output_logits = torch.stack([outputs.start_logits, outputs.end_logits], dim=2)
# print(start,end)

tensor([216, 685, 182, 115], device='cuda:0') tensor([218, 688, 184, 116], device='cuda:0')


In [ ]:
del batch, input_ids, attention_mask, start, end

In [ ]:
print(tokenizer.decode(input_ids[1][549:551]))
print(answer_text[1])

80㎏
80㎏


In [ ]:
# model.to(device)
# outputs = model(input_ids = input_ids, attention_mask = attention_mask,start_positions = start, end_positions = end)
# start_logits = outputs.start_logits[0].softmax(-1)
# end_logits = outputs.end_logits[0].softmax(-1)
# probability = torch.triu(start_logits[:, None] @ end_logits[None, :])
# print(torch.argmax(probability)//len(end_logits)) # 479
# print(torch.argmax(probability)%len(end_logits)) # 493
# print(tokenizer.decode(input_ids[0][48:51]))
# print(answer_text[0])
# print(outputs.loss)

In [ ]:
os.makedirs('dump', exist_ok=True)
train_losses = []
dev_losses = []
step = 0
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

In [ ]:
model.to(device)
for epoch in range(3):
    print("Epoch", epoch)
    # Training
    model.train()
    running_loss = 0.
    losses = []
    progress_bar = tqdm(train_loader, desc='Train')
    for batch, start, end, answer_text in progress_bar:
        input_ids = batch.input_ids.to(device)
        attention_mask = batch.attention_mask.to(device)
        # token_type_ids = batch.token_type_ids.to(device)        
        start = start.to(device)
        end = end.to(device)
        output = model(input_ids = input_ids, attention_mask = attention_mask,start_positions = start, end_positions = end)

        loss = output.loss

        (loss / accumulation).backward()
        # loss.backward()
        running_loss += loss.item()
        del batch, input_ids, attention_mask, start, end, loss

        step += 1
        if step % accumulation:
            continue

        clip_grad_norm_(model.parameters(), max_norm=1.)
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)

        losses.append(running_loss / accumulation)
        running_loss = 0.
        progress_bar.set_description(f"Train - Loss: {losses[-1]:.3f}")

        if step % 512 == 0 :
            # Evaluation
            dev_loss = []
            model.eval()
            for batch, start, end, answer_text in dev_loader:

                input_ids = batch.input_ids.to(device)
                attention_mask = batch.attention_mask.to(device)
                # token_type_ids = batch.token_type_ids.to(device)   
                start = start.to(device)
                end = end.to(device)

                with torch.no_grad():
                    outputs = model(input_ids = input_ids, attention_mask = attention_mask,start_positions = start, end_positions = end)
                loss = outputs.loss

                dev_loss.append(loss.item())
                del batch, input_ids, attention_mask, outputs, loss
            # scheduler.step(mean(losses))
            dev_losses.append(mean(dev_loss))
            print(f"Evaluation score: {dev_losses[-1]:.3f}")
            scheduler.step(mean(dev_loss))
            model.train()
    train_losses.append(mean(losses))
    print(f"train score: {train_losses[-1]:.3f}")

    model.save_pretrained(f'dump/model.{epoch}')

Epoch 0


Train - Loss: 3.030:   9%|▉         | 512/5417 [02:47<19:57:28, 14.65s/it]

Evaluation score: 2.451


Train - Loss: 1.865:  19%|█▉        | 1024/5417 [05:33<17:48:08, 14.59s/it]

Evaluation score: 1.752


...

Train - Loss: 1.435:  57%|█████▋    | 3072/5417 [16:38<9:31:26, 14.62s/it]

Evaluation score: 1.236


Train - Loss: 1.600:  66%|██████▌   | 3584/5417 [19:24<7:26:21, 14.61s/it]

Evaluation score: 1.196


Train - Loss: 1.310:  70%|██████▉   | 3791/5417 [20:12<06:03,  4.47it/s]

In [ ]:
dev_loader = make_dataloader(dev_dataset_custum, 2,'train')
batch, start, end, answer_text = next(iter(dev_loader))
input_ids = batch.input_ids.to(device)
attention_mask = batch.attention_mask.to(device)
# token_type_ids = batch.token_type_ids.to(device)        
start = start.to(device)
end = end.to(device)
print(start, end)

batch_size : 2
tensor([283, 298]) tensor([286, 301])


In [ ]:
model.eval()
outputs = model(input_ids = input_ids, attention_mask = attention_mask,start_positions = start, end_positions = end)
start_logits = outputs.start_logits[1].softmax(-1)
end_logits = outputs.end_logits[1].softmax(-1)
probability = torch.triu(start_logits[:, None] @ end_logits[None, :])
print(torch.argmax(probability)//len(end_logits))
print(torch.argmax(probability)%len(end_logits)) 
# print(probability)
# print(tokenizer.decode(input_ids[0][94:96]))
# print(answer_text[0])

tensor(298)
tensor(301)


In [ ]:
outputs.loss

tensor(1.0031, grad_fn=<DivBackward0>)

# For Test

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("./dump/model.2")
model.to(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "monologg/kobigbird-bert-base",
    use_fast=True # Whether or not to try to load the fast version of the tokenizer.
)

In [ ]:
from typing import List, Tuple, Dict, Any
import json
import random

class TestKoMRC:
    def __init__(self, data, indices: List[Tuple[int, int, int]]):
        self._data = data
        self._indices = indices

    # Json을 불러오는 메소드
    @classmethod
    def load(cls, file_path: str):
        with open(file_path, 'r', encoding='utf-8') as fd:
            data = json.load(fd)

        indices = []
        for d_id, document in enumerate(data['data']):
            for p_id, paragraph in enumerate(document['paragraphs']):
                for q_id, _ in enumerate(paragraph['qas']):
                    indices.append((d_id, p_id, q_id))
        
        return cls(data, indices)

    # 데이터 셋을 잘라내는 메소드
    @classmethod
    def split(cls, dataset, eval_ratio: float=.1, seed=42):
        indices = list(dataset._indices)
        random.seed(seed)
        random.shuffle(indices)
        train_indices = indices[int(len(indices) * eval_ratio):]
        eval_indices = indices[:int(len(indices) * eval_ratio)]

        return cls(dataset._data, train_indices), cls(dataset._data, eval_indices)

    def __getitem__(self, index: int) -> Dict[str, Any]:
        d_id, p_id, q_id = self._indices[index]
        paragraph = self._data['data'][d_id]['paragraphs'][p_id]

        
        qa = paragraph['qas'][q_id]

        guid = qa['guid']
        question = qa['question']
        context = paragraph['context']
        return {
            'guid': guid,
            'context': context,
            'question': question
        }

    def __len__(self) -> int:
        return len(self._indices)

In [ ]:
test_dataset = TestKoMRC.load('./test.json')
print("Number of Test Samples", len(test_dataset))

Number of Test Samples 4008


In [ ]:
next(iter(test_dataset))

{'guid': 'd14cb73158624cf094c546d856fd3c80',
 'context': 'BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션(8대 한정)은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다.',
 'question': '말라카이트

In [ ]:
class TestDataset(Dataset):
    def __init__(self, dataset):
        self.question, self.context, self.guid= self.make_dataset(dataset)

    def make_dataset(self, dataset):
        context, question, guid = [], [], []
        for i, data in enumerate(dataset) :
          context.append(data['context'])
          question.append(data['question'])
          guid.append(data['guid'])
        return question, context, guid
        
    def __len__(self):
        return len(self.question)

    def __getitem__(self, idx):
        return self.question[idx], self.context[idx] , self.guid[idx]

In [ ]:
test_dataset_custum = TestDataset(test_dataset)

In [ ]:
def test_collate_fn(batch):
    global tokenizer
    question_list, context_list, guid_list= [], [], []

    for _question, _context, _guid in batch:
        question_list.append(_question)
        context_list.append(_context)
        guid_list.append(_guid)

    tensorized_input = tokenizer(    
        question_list, context_list,
        add_special_tokens=True,
        padding="longest",  
        max_length=2024,
        truncation="only_second",
        return_offsets_mapping=True,
        return_tensors='pt'
        
    )

    return tensorized_input, guid_list, context_list

In [ ]:
test_loader = DataLoader(test_dataset_custum, batch_size = 2, collate_fn = test_collate_fn)

In [ ]:
input, guid = next(iter(test_loader))
input_ids = input.input_ids.to(device)
attention_mask = input.attention_mask.to(device)

In [ ]:
model.eval()
outputs = model(input_ids = input_ids, attention_mask = attention_mask)
start_logits = outputs.start_logits[0].softmax(-1)
end_logits = outputs.end_logits[0].softmax(-1)
probability = torch.triu(start_logits[:, None] @ end_logits[None, :])
start_token = torch.argmax(probability)//len(end_logits)
end_token = torch.argmax(probability)%len(end_logits)
# print(probability)
print(tokenizer.decode(input_ids[0][start_token:end_token + 1]))
# print(answer_text[0])

뉴 740Li 25주년 에디션 ( 7대 한정 )


In [ ]:
print(tokenizer.decode(input_ids[0]))

[CLS] 말라카이트에서 나온 색깔을 사용한 에디션은? [SEP] BMW 코리아 ( 대표 한상윤 ) 는 창립 25주년을 기념하는 ‘ BMW 코리아 25주년 에디션 ’ 을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션 ( 이하 25주년 에디션 ) 은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린 ( 50대 한정 ) 또는 마카오 블루 ( 50대 한정 ) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션 ( 25대 한정 ) 은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션 ( 25대 한정 ) 은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션 ( 7대 한정 ) 에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션 ( 8대 한정 ) 은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다. [SEP]


In [ ]:
import csv

guid_list = []
infer_list = []
for i,(batch, guid, context) in tqdm(enumerate(test_loader)):
    input_ids = batch.input_ids.to(device)
    attention_mask = batch.attention_mask.to(device)
    offset_mapping = batch.offset_mapping.to(device)

    outputs = model(input_ids = input_ids, attention_mask = attention_mask)
    start_logits = outputs.start_logits.softmax(-1)
    end_logits = outputs.end_logits.softmax(-1)
    
    for guid, start, end, input_ids, offset_mapping, context in zip(guid, start_logits, end_logits, input_ids, offset_mapping, context):
        proba = torch.triu(start[:, None] @ end[None,:])
        start = torch.argmax(proba)//len(end)
        end = torch.argmax(proba)%len(end)
        inference = tokenizer.decode(input_ids[start:end + 1])
        start_charactor = offset_mapping[start][0]
        end_charactor = offset_mapping[end][1]
        inference = context[start_charactor:end_charactor]
        if len(inference) > 30 :
            infer_list.append('무무무무무무')
        else:
            infer_list.append(inference)
        guid_list.append(guid)

2004it [04:12,  7.93it/s]


In [ ]:
test_dataset_custum[7]

('필리핀 숙박업소가 손님을 다시 받으려면 필수적으로 떼야 하는 서류는?',
 '두테르테 필리핀 대통령이 세부를 제외한 전 지역에서 일반 지역사회 격리 조치 (GCQ) 혹은 수정된 일반 지역사회 격리 조치 (MGCQ)를 7월 15일까지 연장함에 따라, 필리핀 관광부는 코로나 예방을 위해, 외식, 교통, 숙박 등 산업 전반에 걸쳐 안전 수칙을 마련하여 실시하고 있다고 밝혔다. 숙박시설의 경우, 필리핀 관광부는 영업을 재개하려는 호텔 및 기타 숙박업소에 의무적으로 ‘운영허가 인증서’를 발급 받도록 하여, 관광객들이 안전하게 숙박시설을 이용할 수 있도록 조치하고 있다. 인증서를 받은 숙박시설에서는 마스크 미 착용시, 체크인이 불가하며, 비 접촉식 체온 확인 후 입실이 가능하다. 또한 로비, 엘리베이터 바닥 등에 1m 간격으로 마크를 부착하여, 사회적 거리 두기를 준수할 수 있게 했다. 그 외에도 객실 앞으로 식사 배달, 전 숙박시설 소독, 직원들의 마스크 착용 의무화 등의 조치로 안전한 휴식을 돕고 있다. 식당 매장 운영은 전체 50%의 가동률로 오후 9시까지 영업이 가능하며, 고객과 직원 간 일정한 거리를 유지해야 한다. 또한 테이블 당 거리가 1m를 유지해야 하며, 뷔페 레스토랑의 경우, 원하는 음식을 주문서에 표시한 뒤, 직원이 접시에 담아 음식을 가져다 주는 형식으로 운영된다. 공공의 위생을 위해, 직원에게 직접 현금을 건네는 접촉 결제는 불가하다. 지역사회 격리조치 기간 동안 대중 교통수단은 점진적으로 허용되고 있다. 이에 따라, 필리핀 관광부는 모든 대중교통 탑승객들에게 마스크를 착용하고 사회적 거리 두기를 준수할 것을 강조했다. 모든 교통수단은 전체 탑승 가능 승객수의 50%만 탑승이 가능하다. 또한 관광객 전용 교통수단의 경우에는, 기본 위생 키트 및 비접촉식 체온계를 구비해야 한다. 현재 외국인들의 필리핀 입국은 외교관 비자 소지자, 필리핀 국적의 외국인 배우자 등으로 제한되고 있으며, 입국 시 시스템에 개인 정보를 기입한 후, 생성 된 QR 코드를

In [ ]:
len(infer_list)

4008

In [ ]:
import pandas as pd
df = pd.DataFrame({'Id' : guid_list,'Predicted' : infer_list})

In [ ]:
df

,Id,Predicted
0,d14cb73158624cf094c546d856fd3c80,뉴 740Li 25주년 에디션
1,906631384e91493ebe1c7f34aea6f241,독일 뒤셀도르프로
2,35e61dcb479643448a2cb7d326ae50a6,링크트인
3,075e761b370040cb9041eecd39afc27c,링크트인과 페이스북
4,e67ed38f3dd944be94d5b4c53731f334,마드리드
...,...,...
4003,05fcb8054dc44dab8683579c2cf5e465,200만엔
4004,cc7f826b66724ce9b39e3a974ca15661,중동 건설 현장
4005,3282034aa41e4fab980851ffd4a868dd,아시아~유럽 노선
4006,0a73550b36df4baf82ac2f98619d22e7,10일


In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/[GOORM]NLP/Project/Machine Reading Comprehension/KoBigbird_base.csv',encoding = 'utf8')